In [25]:
import os
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import torch
from torch import nn
import torch.optim as optim
import torchvision
#pip install torchvision
from torchvision import transforms, models, datasets
#https://pytorch.org/docs/stable/torchvision/index.html
import imageio
import time
import warnings
import random
import sys
import copy
import json
from PIL import Image

### 方法一：采用大幅度的画质压缩到32X32X3

In [26]:
data_dir = 'C:/Users/86135/.cache/torch/self_made_dataset/animals_self_modify_32_32'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'

In [27]:
pic_shape = 32

transforms = transforms.ToTensor()



# train_transforms = transforms.Compose([transforms.RandomRotation(45),#随机旋转，-45到45度之间随机选
#         transforms.CenterCrop(pic_shape),#从中心开始裁剪
#         transforms.RandomHorizontalFlip(p=0.5),#随机水平翻转 选择一个概率概率
#         transforms.RandomVerticalFlip(p=0.5),#随机垂直翻转
#         transforms.ColorJitter(brightness=0.2, contrast=0.1, saturation=0.1, hue=0.1),#参数1为亮度，参数2为对比度，参数3为饱和度，参数4为色相
#         transforms.RandomGrayscale(p=0.025),#概率转换成灰度率，3通道就是R=G=B
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])#均值，标准差
#     ])
# valid_transforms = transforms.Compose([transforms.Resize(256),
#         transforms.CenterCrop(pic_shape),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ])



In [28]:
BATCH_SIZE = 256

train_dataset = torchvision.datasets.ImageFolder(train_dir,transform = transforms)
valid_dataset = torchvision.datasets.ImageFolder(valid_dir,transform = transforms)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset, batch_size=1)

In [29]:
conv_layers_num = 3

in_features = train_dataset[0][0].shape[0]
conv1_features = 32
conv2_features = 64
conv3_features = 128

hidden1_features = conv3_features*(pic_shape/(2**conv_layers_num))**2
hidden2_features = hidden1_features/8
out_feature = len(train_dataset.class_to_idx)
hidden1_features = int(hidden1_features)
hidden2_features = int(hidden2_features)
epochs = 100
lr = 3e-1

In [30]:
hidden2_features

256

In [31]:
class My_model(nn.Module):
    def __init__(self):
        super(My_model,self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=in_features,
                out_channels=conv1_features,
                kernel_size=3,
                stride=1,
                padding=1,
                ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(
                in_channels=conv1_features,
                out_channels=conv2_features,
                kernel_size=3,
                stride=1,
                padding=1,
                ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(
                in_channels=conv2_features,
                out_channels=conv3_features,
                kernel_size=3,
                stride=1,
                padding=1,
                ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )      
        
        self.fc1 = nn.Sequential(
            nn.Linear(hidden1_features,hidden2_features,bias=True),
            nn.ReLU()
        )
        self.fc2 = nn.Sequential(
            nn.Linear(hidden2_features,out_feature,bias=True),
            nn.ReLU()
        )
        self.output = nn.Softmax()
        
    def forward(self,x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
#         print(x.shape)
        x = x.view(x.size(0), -1)
#         print(x.shape)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

model = My_model()
model.to(torch.device("cuda:0"))
optimizer = torch.optim.SGD(model.parameters(),lr = lr)
criterion = nn.CrossEntropyLoss()

In [32]:
model

My_model(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Sequential(
    (0): Linear(in_features=2048, out_features=256, bias=True)
    (1): ReLU()
  )
  (fc2): Sequential(
    (0): Linear(in_features=256, out_features=10, bias=True)
    (1): ReLU()
  )
  (output): Softmax(dim=None)
)

In [33]:
for epoch in range(epochs):
    ## train
    for image,label in train_loader:
        
        image=image.to(torch.device("cuda:0"))
        label=label.to(torch.device("cuda:0")) 
        
        optimizer.zero_grad()
        output = model(image)

        loss = criterion(output,label)
        loss.backward()
        optimizer.step()

#     print("batch finished")
    if((epoch+1)%(epochs/10)==0):
        print("validation!")
        ## test
        groups = 0
        hit = 0
        with torch.no_grad():
            for image,label in valid_loader:
                groups += 1
                
                image=image.to(torch.device("cuda:0"))
                label=label.to(torch.device("cuda:0")) 

                output = model(image)

                predict = np.argmax(output.cpu())

                if(int(predict)==int(label)):
                    hit+=1
        
        print("epoch",epoch+1,"loss =",float(loss),"valid acc = ",(hit/groups))             
    

validation!
epoch 10 loss = 2.0581371784210205 valid acc =  0.2844574780058651
validation!
epoch 20 loss = 1.528288722038269 valid acc =  0.4310850439882698
validation!
epoch 30 loss = 1.5003185272216797 valid acc =  0.46920821114369504
validation!
epoch 40 loss = 0.8452515602111816 valid acc =  0.5032258064516129
validation!
epoch 50 loss = 0.21355901658535004 valid acc =  0.506158357771261
validation!
epoch 60 loss = 0.009838417172431946 valid acc =  0.5372434017595308
validation!
epoch 70 loss = 0.017864612862467766 valid acc =  0.5372434017595308
validation!
epoch 80 loss = 0.0011797869810834527 valid acc =  0.5384164222873901
validation!
epoch 90 loss = 0.0008298491593450308 valid acc =  0.5378299120234604
validation!
epoch 100 loss = 0.00042186910286545753 valid acc =  0.5366568914956011


### 方法二：修改网络结构为主要全连接

In [ ]:
conv_layers_num = 3

in_features = train_dataset[0][0].shape[0]
conv1_features = 32
conv2_features = 64
conv3_features = 128

hidden1_features = conv3_features*(pic_shape/(2**conv_layers_num))**2
hidden2_features = hidden1_features/8
out_feature = len(train_dataset.class_to_idx)
hidden1_features = int(hidden1_features)
hidden2_features = int(hidden2_features)
epochs = 10
lr = 1e-2

In [ ]:
hidden2_features

In [41]:
class My_model(nn.Module):
    def __init__(self):
        super(My_model,self).__init__()
        
        self.fc1 = nn.Linear(32*32*3,hidden2_features,bias=True)
        self.ReLU1= nn.ReLU()
        self.fc2 = nn.Linear(hidden2_features,out_feature,bias=True)
        self.ReLU2= nn.ReLU()
        self.softmax= nn.Softmax()
        
    def forward(self,x):
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.ReLU1(x)
        x = self.fc2(x)
        x = self.ReLU2(x)
        
        output = self.softmax(x)
        return output

model = My_model()
model.to(torch.device("cuda:0"))
optimizer = torch.optim.SGD(model.parameters(),lr = lr)
criterion = nn.CrossEntropyLoss()

In [42]:
model

My_model(
  (fc1): Linear(in_features=3072, out_features=256, bias=True)
  (ReLU1): ReLU()
  (fc2): Linear(in_features=256, out_features=10, bias=True)
  (ReLU2): ReLU()
  (softmax): Softmax(dim=None)
)

In [43]:
for epoch in range(epochs):
    ## train
    for image,label in train_loader:
        
        image=image.to(torch.device("cuda:0"))
        label=label.to(torch.device("cuda:0")) 
        
        optimizer.zero_grad()
        output = model(image)

        loss = criterion(output,label)
        loss.backward()
        optimizer.step()

#     print("batch finished")
    if((epoch+1)%(epochs/10)==0):
        print("validation!")
        ## test
        groups = 0
        hit = 0
        with torch.no_grad():
            for image,label in valid_loader:
                groups += 1
                
                image=image.to(torch.device("cuda:0"))
                label=label.to(torch.device("cuda:0")) 

                output = model(image)

                predict = np.argmax(output.cpu())

                if(int(predict)==int(label)):
                    hit+=1
        
        print("epoch",epoch+1,"loss =",float(loss),"valid acc = ",(hit/groups))             
    

C:\Users\86135\AppData\Local\Temp\ipykernel_17244\1038918849.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(x)


validation!
epoch 10 loss = 2.186697483062744 valid acc =  0.28856304985337244
validation!
epoch 20 loss = 2.161162853240967 valid acc =  0.28328445747800585
validation!
epoch 30 loss = 2.0933990478515625 valid acc =  0.29266862170087976
validation!
epoch 40 loss = 2.025831460952759 valid acc =  0.3460410557184751
validation!
epoch 50 loss = 2.0307528972625732 valid acc =  0.32668621700879763
validation!
epoch 60 loss = 2.070737600326538 valid acc =  0.3530791788856305
validation!
epoch 70 loss = 2.1080455780029297 valid acc =  0.3530791788856305
validation!
epoch 80 loss = 2.002516984939575 valid acc =  0.35835777126099705
validation!
epoch 90 loss = 2.0414140224456787 valid acc =  0.32082111436950145
validation!
epoch 100 loss = 2.02754545211792 valid acc =  0.3161290322580645


In [6]:
import datetime

now = datetime.datetime.now()
print(now)
date_string = now.strftime("%Y_%m_%d_%H_%M_%S")

2023-03-29 15:42:33.383934
当前时间为： 2023_03_29_15_42_33
